# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [249]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = './dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)
df.head()

,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,barrio_match,PH,apartment,house,...,VILLA LUGANO,VILLA LURO,VILLA ORTUZAR,VILLA PUEYRREDON,VILLA REAL,VILLA RIACHUELO,VILLA SANTA RITA,VILLA SOLDATI,VILLA URQUIZA,outlier_price_m2
0,-34.589,-58.417,170000.000,40.000,38.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-34.591,-58.418,90000.000,27.000,27.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-34.587,-58.437,150000.000,44.000,44.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-34.593,-58.428,154000.000,58.000,58.000,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-34.593,-58.428,154000.000,58.000,58.000,3,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [250]:
# Hacé la separación en esta celda
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']

In [251]:
# Esto es lo que hace
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [252]:
# Creá en esta celda la variable param_grid
ran = [1, 2, 3, 4, 5]
param_grid = {'max_depth': ran, 'max_features': ran}
print(param_grid)

{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}


**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [253]:
# Importa y crea un GridSearchCV en esta celda
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(tree, param_grid, scoring='neg_mean_squared_error', return_train_score=True)

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [254]:
# Hace el fit de grid search en esta celda
grid_search.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=42, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'max_features': [1, 2, 3, 4, 5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
          

In [255]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [256]:
grid_search.cv_results_

{'mean_fit_time': array([0.00467854, 0.00263705, 0.00307627, 0.00266986, 0.00282769,
        0.00271697, 0.0025877 , 0.0030766 , 0.003755  , 0.00293794,
        0.0030036 , 0.00299125, 0.00259304, 0.00337777, 0.00307217,
        0.00318704, 0.00354562, 0.00296826, 0.00322719, 0.00342932,
        0.00300488, 0.00309081, 0.00277362, 0.0033596 , 0.00342631]),
 'std_fit_time': array([2.89777046e-03, 1.23875704e-04, 7.63915993e-04, 1.70721115e-04,
        1.51945071e-04, 2.50496288e-04, 8.78807907e-05, 4.64177043e-04,
        6.64319093e-04, 2.41771139e-04, 5.78487395e-04, 1.23100239e-04,
        1.28108672e-05, 3.71937648e-04, 2.59372179e-04, 4.30044959e-04,
        4.31215171e-04, 1.52521521e-04, 9.80928791e-05, 3.19961983e-04,
        5.28432449e-04, 4.85075827e-05, 4.48446524e-05, 1.08731252e-04,
        4.37846409e-04]),
 'mean_score_time': array([0.00151863, 0.00083003, 0.00129991, 0.00093942, 0.0008872 ,
        0.00088234, 0.00092111, 0.0011622 , 0.0008966 , 0.00089393,
        0.00

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [257]:
# Mostrá los grid_scores en esta celda
print(nmsq2rmse(grid_search.cv_results_['mean_train_score']))
print(nmsq2rmse(grid_search.cv_results_['mean_test_score']))

[31244.74 31244.74 31244.74 31244.74 31244.74 31219.27 31185.99 31210.3
 30937.25 31161.14 31161.32 30867.66 31184.46 30839.89 29878.98 30992.83
 30747.02 30993.4  29207.41 26483.19 30926.48 29215.51 30881.71 28902.97
 28432.7 ]
[31258.55 31258.55 31258.55 31258.55 31258.55 31235.98 31254.6  31238.92
 30989.93 31213.9  31219.38 30984.42 31293.28 30923.09 30017.78 31084.46
 30904.49 31200.79 29436.15 26737.13 31132.51 29535.17 31082.75 29260.13
 28777.03]


De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

### El ejercicio marca que los mejores resultados son con `max_depth` 3 y `max_features` 3 y no es asi

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [258]:
# Mostrás los resultados en esta celda

print(nmsq2rmse(grid_search.best_score_))
print(grid_search.best_params_)

26737.13
{'max_depth': 4, 'max_features': 5}


#### Los mejores resultados son con `max_depth` 4 y `max_features` 5

Convertimos a RMSE.

In [259]:
from sklearn.metrics import mean_squared_error
y_opt_pred = grid_search.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

26358.0

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [260]:
param_grid = {
    'min_samples_split': [2, 10, 20],
    'max_depth': [None, 2, 5, 10, 15],
    "min_samples_leaf": [1, 5, 10, 15],
    "max_leaf_nodes": [None, 5, 10, 20]
}
tree = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(tree, param_grid, scoring='neg_mean_squared_error', return_train_score=True, refit=True)
grid_search.fit(X_train,y_train)
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [261]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [262]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [263]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [264]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
